# Explore Oura User Data with Elaticsearch & Kibana

Oura Documentation for the Developer API: https://cloud.ouraring.com/v2/docs 

In [ ]:
! pip install -r requirements.txt
import requests 
import json
from elasticsearch import Elasticsearch, helpers
from getpass import getpass

Generating an auth token for your account: https://cloud.ouraring.com/docs/authentication

In [44]:
auth_token = getpass("auth token: ")

In [43]:
def get_data(type):
  url = 'https://api.ouraring.com/v2/usercollection/' + type
  params={ 
      'start_date': '2021-11-01', 
      'end_date': '2025-01-01' 
  }
  headers = { 
    'Authorization': 'Bearer ' + auth_token 
  }
  response = requests.request('GET', url, headers=headers, params=params) 
  return response.json()["data"]

In [40]:
data = get_data("daily_activity")
with open('oura_data_activity.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [41]:
data = get_data("sleep")
with open('oura_data_sleep.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

## Get the data into elastic

In [53]:
#Connect to the elastic cloud server
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")
ELASTIC_API_KEY = getpass("Elastic API Key: ")

# Create an Elasticsearch client using the provided credentials
client = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,  # cloud id can be found under deployment management
    api_key=ELASTIC_API_KEY, # your username and password for connecting to elastic, found under Deplouments - Security
)

In [ ]:
index_name = 'oura-history-sleep'

# Create the Elasticsearch index with the specified name (delete if already existing)
if client.indices.exists(index=index_name):
    client.indices.delete(index=index_name)
client.indices.create(index=index_name)

with open("oura_data_sleep.json", "r") as f:
    json_data = json.load(f)
    documents = []
    for doc in json_data:
        documents.append(doc)
    load = helpers.bulk(client, documents, index=index_name)

We can now search our data:

In [59]:
index_name = 'oura-history-sleep'

response = client.search(index=index_name, size=10)

print("We get back {total} results, here are the first ones:".format(total=response["hits"]['total']['value']))
for hit in response["hits"]["hits"]:
    print("Heart rate average: {hra} and sleeping score: {score}".format(hra=hit['_source']['average_heart_rate'], score= hit['_source']['readiness']['score']))

We get back 137 results, here are the first ones:
Heart rate average: 62.5 and sleeping score: 89
Heart rate average: 62.5 and sleeping score: 57
Heart rate average: 65.25 and sleeping score: 80
Heart rate average: 62.375 and sleeping score: 91
Heart rate average: 64.625 and sleeping score: 81
Heart rate average: 66.375 and sleeping score: 46
Heart rate average: 62.875 and sleeping score: 84
Heart rate average: 62.25 and sleeping score: 87
Heart rate average: 63.75 and sleeping score: 84
Heart rate average: 63.75 and sleeping score: 92
